<img src="../materials/title.png">

# <center><font color='#58ACFA'> AI Workshop - Część I</font></center>
## <center>Wykorzystanie generatywnych autoenkoderów do ekstrakcji cech istotnych w procesie wyszukiwania podobnych twarzy</center>

## <font color='#58ACFA'>Plan pracy:</font>

#### <font color='#0B3861'> >>> Zbudujemy autoenkoder wariacyjny zdolny do identyfikacji charakterystycznych cech twarzy.</font>
#### <font color='#0B3861'> >>> Wykorzystamy zbudowany autoenkoder do odnajdywania podobnych twarzy.</font>
#### <font color='#0B3861'> >>> Stworzymy nowe twarze :D</font>


### <font color='#0B3861'> Do tego celu posłużymy się danymi pochodzącymi ze zbioru CelebA. Zbiór ten posiada ponad 200 tysięcy zdjęć twarzy znanych celebrytów.</font>

<img src="../materials/dataset.png" width="1000">

### <font color='red'>Uwaga: Miejsca w kodzie do uzupełnienia są oznaczone poprzez "____"</font>

## <font color='#0B3861'>Zaczniemy od zaimportowania niezbędnych bibliotek i ustawienia odpowiednich folderów</font>
##### Wykorzystane funkcje zostaną wytłumaczone w dalszej części notatnika

In [ ]:
!pip install --user -r ../requirements.txt # pobieranie bibliotek z pliku requirements.txt

In [ ]:
# operacje związane ze ścieżkami
import os
from glob import glob

# operacje na macierzach
import numpy as np

# operacje na obrazach
from PIL import Image

# wizualizacja postępu wykonania kodu
from tqdm import tqdm

# tworzenie wykresów i wizualizacja danych
import matplotlib.pyplot as plt

# importy związane z budową i treningiem modelu
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input 
from keras.layers import Conv2D, Flatten, Lambda
from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, kullback_leibler_divergence
from keras.utils import plot_model
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint

# importy związane z wyszukiwaniem podobnych twarzy
from sklearn.metrics import pairwise_distances

%matplotlib inline

## <font color='#0B3861'>Pobranie i rozpakowanie danych</font>

*   <font color='red'>Linux</font>

In [ ]:
!wget https://www.dropbox.com/s/lpmjzzk26nae9bh/img_align_celeba.zip
!unzip -qq img_align_celeba.zip -d ..

*   Windows  
Wejdź na stronę: https://www.dropbox.com/s/lpmjzzk26nae9bh/img_align_celeba.zip
Pobierz plik i wypakuj w utworzonym folderze (*img_align_celeba*)

In [ ]:
# tworzymy foldery na zdjęcia i modele
dataset_dir = '../img_align_celeba'
model_dir = '../models'

os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

#### Zawsze warto ustawić tak zwane ziarno - określa ono punkt początkowy dla algorytmu generacji liczb pseudolosowych. Dzięki temu przy każdorazowym uruchomieniu notatnika uzyskamy te same wyniki. 
Podpowiedź: ziarno ustawiamy podając dowolną liczbę do funkcji <font color=red>np.random.seed()</font>

In [ ]:
# ustaw swoje ziarno
____

## <font color='#0B3861'>Wczytanie i przygotowanie danych</font>
#### Dane powinny znajdować się w folderze *dataset_dir*

In [ ]:
img_paths = glob(dataset_dir + "/*.jpg") # pobierze ścieżki do każdego z obrazów w folderze

#### Wszystkich obrazów powinno być 202599. Sprawdź, czy rzeczywiście tak jest i nic się nie zgubiło :)

In [ ]:
# wyświetl ilość obrazów wczytanych do notatnika
____

#### W celu przyspieszenia obliczeń wykorzystamy jedynie 10 000 losowo wybranych obrazów. Potrzebujemy zatem 10 000 losowo wybranych indeksów. Pozwala na to funkcja <font color=red>np.random.choice()</font>, która przyjmuje dwa argumenty: pierwszy określa elementy z jakich losujemy (w naszym przypadku będzie to liczba wszystkich obrazów), a drugi to ilość liczb jakie zostaną zwrócone
##### *ustawione wcześniej ziarno sprawi, że zawsze wylosujemy ten sam zestaw liczb

In [ ]:
# Wygeneruj 10000 losowych indeksów
rnd_choice = _____

#### Razem z wczytaniem danych  (<font color='red'>Image.open()</font>) przygotujemy je do wykorzystania: przytniemy krawędzie (<font color='red'>crop()</font>) i zmniejszymy ich rozmiar (<font color='red'>resize()</font>), co znacznie przyspieszy trening modelu.
#### Zastosuj wylosowane indeksy w celu wczytania zbioru danych do pracy. Przytnij obraz wprowadzając wartości 9, 13, 169 i 205 (koordynaty pikseli) w odpowednie miejsce. Następnie zmniejsz obraz do rozmiaru 80x96

In [ ]:
images = np.array(
                [np.array(Image.open(img_paths[x]).crop(box=(____)).resize((____)))
                 for x in tqdm(rnd_choice.tolist())]
                ).astype(np.float32)

#### Żeby upewnić się, że wszystko poszło dobrze, wyświetl wymiary obrazów 

In [ ]:
____

#### Dokończ skalowanie obrazów do wartości z przedziału [0,1], w każdym z kanałów RGB (3), wykorzystując funkcje biblioteki numpy (<font color='red'>np.min(), np.max()</font>) oraz poniższy wzór:
\begin{equation}
X = \frac{X - min(X)}{max(X) - min(X)}
\end{equation}

In [ ]:
for i in range(3):
    images[:,:,:,i] = (____ - ____)/(____ - ____)

#### Upewnij się, że z danymi wszystko w porządku zanim przejdziesz do ich analizy. Wyświetl przykładowe 9 obrazów

In [ ]:
plt.figure(figsize=(10,10))
for i in range(1,10):
    plt.subplot(3,3,i)
    plt.imshow(np.squeeze(images[i]))
    plt.axis('off')

## <font color='#0B3861'>Budowa modelu wariacyjnego autoenkodera (VAE)</font>

#### Na początku zdefiniujemy parametry określające model VAE i jego trening

In [ ]:
""" Parametry określające model """
kernel_size = 3 # zwyczajowa wielkość fitra konwolucyjnego
filters = 16 # liczba filtrów dla pierwszej warstwy konwolucyjnej, rośnie 2-krotnie dla następnych
layers = 3 # ilość warstw konwolucyjnych tworzących enkoder i dekoder
input_shape = images.shape[1:4] # wielkość obrazu wejściowego, wykorzystywana dla budowy grafu SN
n_dense = 256 # liczba neuronów dla środkowej warstwy ukrytej - perceptronu
latent_dim = 32 # liczba cech które uzyskamy z enkodera

""" Parametry określające trening sieci """
batch_size = 64 # liczba obrazów wykorzystywana do jednego kroku treningu SN
optimizer = 'adam' # funkcja odpowiadająca za redukcję błędu sieci
epochs = 30 # maksymalna liczba epok, czyli ile razy sieć będzie widziała dane treningowe
model_path = os.path.join(model_dir,'celeb_vae_v2.h5') # ustawienie ścieżki do pliku z modelem

#### Zdefiniujemy niezbędne funkcje: <font color='red'>sampling()</font> generującą nowe dane w warstwie ukrytej *z* oraz <font color='red'>vae_loss()</font> będącą funkcją straty wykorzystującą błąd MAE jak i dywergencję Kullbacka-Leiblera

In [ ]:
def sampling(args):
    """ Funkcja generująca wartości ze standardowych rozkładów normalnych """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
def vae_loss():
    """ Funkcja strat (MSE + Kullback-Leibler) """
    
    reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= input_shape[0] * input_shape[1]
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    
    return vae_loss

### Na podstawie poniższego schematu i wskazówek zbuduj model wariacyjnego autoenkodera (uzupełnij brakujące części kodu) 

<img src="../materials/vae.png" width="1000">

<img src="../materials/vae_legenda.png" width="400">

### <font color='red'> Ważne wskazówki </font>:
*   Zwróć uwagę na kolory warstw na powyższym schemacie i nazwy w nawiasach
*   Brakujące warstwy uzupełniaj analogicznie do wpisanych już przykładów
*   Uzupełnij brakujące pola "____" w wyznaczonych miejscach
*   Funkcja aktywacji w warstwach konwolucyjnych to <font color='red'>'relu'</font>
*   Dla każdej kolejnej warstwy konwolucyjnej (W ENKODERZE!) liczbę filtrów <font color='red'>zwiększ dwukrotnie w stosunku do poprzedniej</font>
*   Dla każdej kolejnej warstwy konwolucyjnej transponowanej (W DEKODERZE!) liczbę filtrów <font color='red'>zmniejsz dwukrotnie w stosunku do poprzedniej</font>
*   Stosujemy padding z zachowaniem wymiarów (<font color='red'>'same'</font>)

In [ ]:
# ENKODER

inputs = Input(shape=input_shape, name='encoder_input') # stworzenie wejścia sieci neuronowej
x = inputs

x = Conv2D(filters=filters*2, kernel_size=kernel_size, 
           activation='relu', strides=2, padding='same')(x) #pierwsza warstwa konwolucyjna enkodera




# TU DODAJ BRAKUJĄCE WARSTWY: x = ___ itp, ze zwiększoną DWUKROTNIE liczbą filtrów w stosunku do poprzedniej warstwy (*4, *8)
x = ____(____)(x)
x = ____(____)(x)



shape = K.int_shape(x) # zapisanie wielkości wyjścia warstwy do późniejszego stworzenia dekodera

x = ____()(x) #SPŁASZCZENIE danych do formy wektora
x = Dense(n_dense, activation=____)(x) #dodatkowa pośrednia warstwa pełna

# trenowalne średnie oraz wariancje do celu generacji wektora ukrytego
z_mean = ____(latent_dim, name='z_mean')(x)
z_log_var = _____(latent_dim, name='z_log_var')(x)


z = ____(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var]) # wartości wyjściowe z enkodera

encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder') # stworzenie enkodera



# DEKODER

latent_inputs = Input(shape=(latent_dim,), name='z_sampling') # stworzenie wejścia dla dekodera


x = ____(n_dense, activation=____)(latent_inputs) # pośrednia warstwa pełna


x = Dense(shape[1] * shape[2] * shape[3], activation=____)(x)
x = Reshape((shape[1], shape[2], shape[3]))(x) # wektor -> macierz


x = Conv2DTranspose(filters=filters*4, kernel_size=____, 
                    activation=____, strides=2, padding=____)(x) #pierwsza warstwa konwolucyjna dekodera



# TU DODAJ BRAKUJĄCE WARSTWY: x = ___ itp, ze zmniejszoną DWUKROTNIE liczbą filtrów w stosunku do poprzedniej warstwy (*2, *1)
x = ____(____)(x)
x = ____(____)(x)


outputs = _____(filters=3, kernel_size=kernel_size, activation='sigmoid', 
                padding='same', name='decoder_output')(x) # warstwa wyjściowa


decoder = Model(____, ____, name='decoder') # stworzenie dekodera (wpisz dane wejściowe i wyjściowe z dekodera)


# stworzenie modelu VAE

outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae')

## <font color='#0B3861'>Trening modelu wariacyjnego autoenkodera (VAE)</font>

### Przygotowanie do treningu
#### Podział danych na treningowe i testowe

In [ ]:
X_train, X_test = train_test_split(images, test_size = 0.2)

#### Wprowadzenie parametrów treningowych i funkcji monitorujących przebieg treningu

In [ ]:
vae.add_loss(vae_loss())
vae.compile(optimizer=optimizer)

early_stop = EarlyStopping(restore_best_weights=True, patience=3) # zatrzyma uczenie jeśli nie będzie poprawy przez 3 epoki
checkpt = ModelCheckpoint(model_path, save_best_only=True, save_weights_only=True) # zapisze wagi dla najlepszej epoki

### Trening

In [ ]:
history = vae.fit(X_train,
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(X_test, None),
                  verbose=2,
                  callbacks = [early_stop, checkpt])

## <font color='#0B3861'>Weryfikacja rezultatów</font>

#### Wczytanie zapisanych wag z epoki, dla której wartość funkcji straty była najniższa

In [ ]:
vae.load_weights(model_path)

#### Jeśli zastosujemy .summary() na utworzonym modelu, uzyskamy informacje o jego budowie. Wyświetl budowę enkodera oraz dekodera oraz odpowiedz na pytania

In [ ]:
print(____.summary()) # budowa enkodera

print(____) # budowa dekodera

#### Jak zmienił się kształt obrazów wejściowych po pierwszej warstwie konwolucyjnej enkodera? : _____
#### Jaka jest całkowita liczba trenowalnych parametrów w dekoderze? : _____
#### Jakie są wymiary filtrów wychodzących z drugiej warstwy konwolucyjnej transponowanej dekodera? : _____
#### Dlaczego w podanych wymiarach na każdej z warstw na pierwszej pozycji pojawia się None? : _____

### Dokonaj predykcji danych testowych na enkoderze i dekoderze (wyjściowa macierz <font color='red'>z</font> z enkodera jest wejściem do dekodera):

In [ ]:
z_mean, z_std_var, z = encoder.predict(X_test)

y_pred = decoder.____(____)

#### Wyświetl wymiary wyjściowej macierzy z. Czym są wiersze a czym kolumny tej macierzy ?

In [ ]:
# wymiary macierzy z:
____

### Wyświetlenie zdjęć oryginalnych i wyjściowych z autoenkodera

In [ ]:
fig, ax = plt.subplots(10,2, figsize = (10,45))
for i in range(10):
    ax[i,0].imshow(np.squeeze(X_test[i]))
    ax[i,0].set_title('Prawdziwe zdjęcie')
    ax[i,0].axis('off')
    ax[i,1].imshow(np.squeeze(y_pred[i]))
    ax[i,1].set_title('Wyjście z VAE')
    ax[i,1].axis('off')
plt.show()

## <font color='#0B3861'>Wyszukiwanie twarzy najbardziej do siebie podobnych</font>

### Do wybranego przez siebie zdjęcia twarzy odnajdź 5 najbardziej podobnych testowych przykładów.
#### Wybierz zdjęcie i wyświetl je:

In [ ]:
# wybierz jeden obraz z danych testowych (X_test) poprzez indeksowanie

index = ____ # wybierz numer przykładu
ref = ____ # wyciągnij obraz o indeksie 'index' z danych testowych

plt.imshow(np.squeeze(ref))
plt.axis('off')
plt.show()

#### Dla wybranego obrazu wyświetl jego wersję wychodzącą z dekodera:

In [ ]:
y_pred_ref = y_pred[index]

plt.imshow(np.squeeze(y_pred_ref))
plt.axis('off')
plt.show()

### Wylicz odległość pomiędzy wybranym obrazem a pozostałymi przykładami testowymi. Wyliczony wektor odległości posortuj rosnąco i wybierz 5 najniższych indeksów - oznaczających najbardziej podobne przykłady do wybranego przez ciebie

#### Odległość jest wyliczana pomiędzy wyjściowymi wektorami cech z enkodera (odległość między wektorami <font color='red'>z</font> wybranego obrazu a pozostałych przykładów). Wykorzystamy odległość euklidesową (<font color='red'>euclidean</font>), ale możesz poeksperymentować z innymi metrykami (np. manhattan, correlation, sqeuclidean czy cosine)

In [ ]:
z_ref = z[index]
z_ref = z_ref.reshape(1, -1) # konwersja na macierz z jedną obserwacją (1 wiersz)

distances = pairwise_distances(____, _____, metric='____')

# distances to macierz zawierająca obliczone odległości pomiędzy wybranym obrazem (wektor cech) a każdym przykładem testowym
print(distances.shape) 

#### Mając policzoną macierz odległości, posortuj ją rosnąco i wybierz 5 pierwszych indeksów. Wykorzystaj do tego celu funkcję z biblioteki numpy <font color='red'>np.argsort()</font>

In [ ]:
indices_sorted = _____(____)[0] # uzyskaj indeksy posortowanych wartości

closest = indices_sorted[:____] # wybierz 5 najbliższych

### Wyświetl odnalezione, najbardziej podobne obrazy
#### Najbliższe obrazy uzyskane z dekodera

In [ ]:
for i in closest:
    plt.imshow(np.squeeze(y_pred[i]))
    plt.axis('off')
    plt.show()

#### Najbliższe obrazy oryginalne

In [ ]:
for i in closest:
    plt.imshow(np.squeeze(X_test[i]))
    plt.axis('off')
    plt.show()

### Dyskusja:
### <font color='#58ACFA'>>>> Które obrazy są do siebie bardziej podobne (uzyskane z dekodera czy oryginalne wersje)?</font>
### <font color='#58ACFA'>>>> Co jest przyczyną takiego rezultatu ?</font>
### <font color='#58ACFA'>>>> Czy podobieństwa pomiędzy obrazami są wyraźnie widoczne ? Jeśli nie, jaka może być tego przyczyna ?</font>

## <font color='#0B3861'>Generowanie nowych twarzy</font>

### VAE to model generatywny, co znaczy, że pozwala na stworzenie nowych, nieistniejących przykładów
#### Polega to na tym, że zamiast wprowadzać do dekodera wyjściowy wektor cech z enkodera, wprowadzamy wektory z losowo wybranymi liczbami próbkowanymi z rozkładu standardowego (średnia równa 0, odchylenie standardowe równe 1)
#### Możemy to osiągnąć za pomoca funkcji <font color='red'>np.random.randn()</font>, która zwraca losowe liczby z rozkładu N(0,1). Funkcja przyjmuje wartości określające rozmiar generowanej macierzy. W naszym przypadku musi być to wektor o długości wektora cech, wychodzącego z enkodera, dla jednego obrazu (jeden wiersz, ilość cech)

In [ ]:
fig, ax = plt.subplots(5,4, figsize = (10,10))
for i in range(5):
    for j in range(4):
        z_generated = np.random.randn(____,____) # stwórz wektor losowych liczb o długości wektora cech
        y_pred_generated = ____.predict(____) # uzyskaj predykcję z dekodera na wygenerowanym wektorze
        ax[i,j].imshow(np.squeeze(y_pred_generated))
        ax[i,j].axis('off')
plt.show()

### Czy wygenerowane twarze wyglądają na rzeczywiste? :)